<a href="https://colab.research.google.com/github/jniplig/ReportKommentz/blob/main/ReportKommentz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Report Comments Project MVP
# ===========================

import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import re
import os
from google.colab import files
import io

# Global variables to store our data
student_data = None
comment_segments = None
selected_teacher = None
selected_term = None
selected_units = []
current_student_index = 0
student_comments = {}  # To store the final comments for each student

In [10]:
# Helper functions for pronoun replacement
def replace_pronouns(text, gender):
    """Replace pronoun placeholders based on gender."""
    if gender == 'M':
        replacements = {
            '[he/she]': 'he',
            '[He/She]': 'He',
            '[his/her]': 'his',
            '[His/Her]': 'His',
            '[him/her]': 'him',
            '[himself/herself]': 'himself'
        }
    elif gender == 'F':
        replacements = {
            '[he/she]': 'she',
            '[He/She]': 'She',
            '[his/her]': 'her',
            '[His/Her]': 'Her',
            '[him/her]': 'her',
            '[himself/herself]': 'herself'
        }
    else:  # Default to male if gender is not specified or is something else
        replacements = {
            '[he/she]': 'he',
            '[He/She]': 'He',
            '[his/her]': 'his',
            '[His/Her]': 'His',
            '[him/her]': 'him',
            '[himself/herself]': 'himself'
        }

    for placeholder, replacement in replacements.items():
        text = text.replace(placeholder, replacement)

    return text

def replace_firstname(text, firstname):
    """Replace firstname placeholder with student's name."""
    text = text.replace('[First_Name]', firstname)
    return text

def personalize_comment(text, firstname, gender):
    """Apply both firstname and pronoun replacements."""
    text = replace_firstname(text, firstname)
    text = replace_pronouns(text, gender)
    return text
    # Move this function to appear BEFORE upload_student_data

def on_teacher_selected(change):
    """Handle teacher selection."""
     print("Teacher selection function called!")
     global selected_teacher
    selected_teacher = change.new

    # Filter students for this teacher
    teacher_students = student_data[student_data['Teacher'] == selected_teacher]

    print(f"\nSelected teacher: {selected_teacher}")
    print(f"This teacher has {len(teacher_students)} students.")

    # Create sample comments if they don't exist
    if comment_segments is None:
        create_sample_comments()

    # Display term selection
    show_term_selection()

In [11]:
# UI functions
def upload_student_data():
    """Upload and process student data."""
    clear_output()
    print("Please upload the student data Excel file...")

    uploaded = files.upload()

    if not uploaded:
        print("No file was uploaded.")
        return

    filename = list(uploaded.keys())[0]
    content = uploaded[filename]

    global student_data
    student_data = pd.read_excel(io.BytesIO(content))

    # Display a summary of the data
    print(f"Loaded {len(student_data)} student records.")
    print("\nFirst 5 records:")
    display(student_data.head())

    # Extract unique teachers
    teachers = student_data['Teacher'].unique()
    teacher_dropdown = widgets.Dropdown(
        options=teachers,
        description='Select Teacher:',
        style={'description_width': 'initial'}
    )

    display(teacher_dropdown)
    def on_dropdown_change(change):
    if change['name'] == 'value':
        on_teacher_selected(change)

teacher_dropdown.observe(on_dropdown_change)

def create_sample_comments():
    """Create sample comment segments for testing."""
    global comment_segments

    # Create a dataframe with sample comment segments
    data = {
        'Category': ['Intro', 'Intro', 'Effort', 'Effort', 'Attitude', 'Attitude', 'Closing', 'Closing'],
        'Code': ['INT01', 'INT02', 'EFF01', 'EFF02', 'ATT01', 'ATT02', 'CLS01', 'CLS02'],
        'Text': [
            '[First_Name] has made a good start to the term in PE.',
            '[First_Name] has shown excellent progress in PE this term.',
            '[He/She] consistently puts forth good effort in lessons.',
            '[First_Name] demonstrates enthusiasm and commitment in all activities.',
            '[He/She] has a positive attitude toward learning new skills.',
            '[First_Name] shows determination when facing challenges in PE.',
            'I am pleased with [his/her] progress and look forward to seeing further development.',
            '[First_Name] should continue to practice regularly to build [his/her] confidence further.'
        ],
        'Level': [1, 2, 1, 2, 1, 2, 1, 2],
        'Unit': ['General', 'General', 'General', 'General', 'General', 'General', 'General', 'General']
    }

    comment_segments = pd.DataFrame(data)
    print("Created sample comment segments for testing:")
    display(comment_segments)

    # Add some more sports-specific comments
    swimming_data = {
        'Category': ['Swimming', 'Swimming', 'Swimming'],
        'Code': ['SWM01', 'SWM02', 'SWM03'],
        'Text': [
            '[First_Name] is developing confidence in the water.',
            '[He/She] can swim independently and is learning different strokes.',
            '[First_Name] shows excellent technique in [his/her] swimming.'
        ],
        'Level': [1, 2, 3],
        'Unit': ['Swimming', 'Swimming', 'Swimming']
    }

    football_data = {
        'Category': ['Football', 'Football', 'Football'],
        'Code': ['FTB01', 'FTB02', 'FTB03'],
        'Text': [
            '[First_Name] is learning the basic rules and skills of football.',
            '[He/She] participates well in team games and is developing ball control.',
            '[First_Name] shows talent in football and demonstrates good tactical awareness.'
        ],
        'Level': [1, 2, 3],
        'Unit': ['Football', 'Football', 'Football']
    }

    # Append to the main dataframe
    comment_segments = pd.concat([
        comment_segments,
        pd.DataFrame(swimming_data),
        pd.DataFrame(football_data)
    ], ignore_index=True)

    print("\nAdded sport-specific comments:")
    display(comment_segments[comment_segments['Unit'] != 'General'])

IndentationError: expected an indented block after function definition on line 33 (<ipython-input-11-c6225d60dd90>, line 34)

In [4]:
def start_comment_creation():
    """Begin the comment creation process."""
    global current_student_index
    current_student_index = 0

    # Get filtered students for this teacher
    teacher_students = student_data[student_data['Teacher'] == selected_teacher].reset_index(drop=True)

    # If there are students, show the first one
    if len(teacher_students) > 0:
        show_student_comment_builder(teacher_students)
    else:
        print("No students found for this teacher.")

def show_student_comment_builder(filtered_students):
    """Show the comment builder for the current student."""
    clear_output()

    if current_student_index >= len(filtered_students):
        # All students have been processed
        show_final_comments(filtered_students)
        return

    # Get current student
    student = filtered_students.iloc[current_student_index]

    # Display student info
    print(f"Teacher: {selected_teacher}")
    print(f"Term: {selected_term}")
    print(f"Units: {', '.join(selected_units)}")
    print("\n" + "="*50)
    print(f"Student {current_student_index + 1} of {len(filtered_students)}")
    print(f"Name: {student['Forename (Firstname)']} {student['Surname']}")
    print(f"Class: {student['Form']}")
    print(f"Gender: {student['Gender']}")
    print("="*50 + "\n")

    # Filter comment segments by selected units
    filtered_segments = comment_segments[
        (comment_segments['Unit'].isin(selected_units)) |
        (comment_segments['Unit'] == 'General')
    ]

    # Group by category
    categories = filtered_segments['Category'].unique()

    # Create accordion for each category
    accordion = widgets.Accordion()

    # Store all checkboxes to access them later
    all_checkboxes = {}

    # Create content for each category
    children = []
    for category in categories:
        category_segments = filtered_segments[filtered_segments['Category'] == category]

        # Create a VBox for this category
        category_box = widgets.VBox()
        category_children = []

        # Add checkboxes for each comment in this category
        for _, segment in category_segments.iterrows():
            # Preview the personalized comment
            firstname = student['Preferred Name'] if pd.notna(student['Preferred Name']) else student['Forename (Firstname)']
            personalized = personalize_comment(segment['Text'], firstname, student['Gender'])

            # Create checkbox with the personalized comment
            checkbox = widgets.Checkbox(
                value=False,
                description=f"{segment['Code']}: {personalized}",
                disabled=False,
                layout=widgets.Layout(width='90%')
            )

            # Store the checkbox with a unique key
            all_checkboxes[segment['Code']] = {
                'checkbox': checkbox,
                'text': segment['Text'],
                'category': category
            }

            category_children.append(checkbox)

        category_box.children = category_children
        children.append(category_box)

    accordion.children = children

    # Set the titles
    for i, category in enumerate(categories):
        accordion.set_title(i, category)

    # Create text area for preview
    preview_area = widgets.Textarea(
        value='',
        placeholder='Selected comments will appear here...',
        description='Preview:',
        disabled=True,
        layout=widgets.Layout(width='90%', height='150px')
    )

    # Create ordered list for selected comments
    selected_list = widgets.SelectMultiple(
        options=[],
        description='Selected:',
        disabled=False,
        layout=widgets.Layout(width='90%', height='150px')
    )

    # Function to update the preview
    def update_preview():
        selected_codes = [code for code, item in all_checkboxes.items() if item['checkbox'].value]

        if not selected_codes:
            preview_area.value = ''
            selected_list.options = []
            return

        # Update the selected list
        selected_list.options = selected_codes

        # Create the preview
        firstname = student['Preferred Name'] if pd.notna(student['Preferred Name']) else student['Forename (Firstname)']
        gender = student['Gender']

        preview_text = []
        for code in selected_codes:
            comment_text = all_checkboxes[code]['text']
            personalized = personalize_comment(comment_text, firstname, gender)
            preview_text.append(personalized)

        preview_area.value = ' '.join(preview_text)

    # Observe all checkboxes
    for item in all_checkboxes.values():
        item['checkbox'].observe(lambda change: update_preview(), names='value')

    # Buttons for navigation
    prev_button = widgets.Button(
        description='Previous Student',
        disabled=current_student_index == 0,
        button_style='info'
    )

    submit_button = widgets.Button(
        description='Submit Comments',
        button_style='success'
    )

    # Button actions
    def on_prev_clicked(b):
        global current_student_index
        current_student_index -= 1
        show_student_comment_builder(filtered_students)

    def on_submit_clicked(b):
        # Get selected comments
        selected_codes = [code for code, item in all_checkboxes.items() if item['checkbox'].value]

        if not selected_codes:
            print("Please select at least one comment before submitting.")
            return

        # Create the final comment
        firstname = student['Preferred Name'] if pd.notna(student['Preferred Name']) else student['Forename (Firstname)']
        gender = student['Gender']

        final_text = []
        for code in selected_codes:
            comment_text = all_checkboxes[code]['text']
            personalized = personalize_comment(comment_text, firstname, gender)
            final_text.append(personalized)

        final_comment = ' '.join(final_text)

        # Store the comment for this student
        student_id = student['Surname'] + '_' + student['Forename (Firstname)']
        student_comments[student_id] = {
            'SchoolID': student.get('SchoolID', ''),
            'Surname': student['Surname'],
            'Forename': student['Forename (Firstname)'],
            'Form': student['Form'],
            'Gender': student['Gender'],
            'Comment': final_comment
        }

        # Move to the next student
        global current_student_index
        current_student_index += 1
        show_student_comment_builder(filtered_students)

    prev_button.on_click(on_prev_clicked)
    submit_button.on_click(on_submit_clicked)

    # Layout everything
    display(widgets.VBox([
        accordion,
        widgets.HBox([selected_list, preview_area]),
        widgets.HBox([prev_button, submit_button])
    ]))

In [5]:
def show_final_comments(filtered_students):
    """Show all final comments and export options."""
    clear_output()

    print(f"Teacher: {selected_teacher}")
    print(f"Term: {selected_term}")
    print(f"Units: {', '.join(selected_units)}")
    print("\nAll comments have been created! Here's a summary:\n")

    # Create a dataframe from the student comments
    comments_df = pd.DataFrame.from_dict(student_comments, orient='index')

    display(comments_df)

    # Button to export to ISAMS format
    export_button = widgets.Button(
        description='Export to iSAMS Format',
        button_style='success'
    )

    def on_export_clicked(b):
        export_to_isams_format(comments_df)

    export_button.on_click(on_export_clicked)

    # Button to start over
    restart_button = widgets.Button(
        description='Start Over',
        button_style='warning'
    )

    def on_restart_clicked(b):
        # Reset the global variables
        global selected_teacher, selected_term, selected_units, current_student_index, student_comments
        selected_teacher = None
        selected_term = None
        selected_units = []
        current_student_index = 0
        student_comments = {}

        # Restart from teacher selection
        upload_student_data()

    restart_button.on_click(on_restart_clicked)

    display(widgets.HBox([export_button, restart_button]))

def export_to_isams_format(comments_df):
    """Export the comments to iSAMS format."""
    clear_output()

    print("Preparing iSAMS export format...")

    try:
        # Ask user to upload the iSAMS template
        print("Please upload the iSAMS template Excel file...")

        uploaded = files.upload()

        if not uploaded:
            print("No file was uploaded.")
            return

        filename = list(uploaded.keys())[0]
        content = uploaded[filename]

        # Load the workbook
        wb = pd.ExcelFile(io.BytesIO(content))

        # Find sheets that correspond to classes/sets
        class_sheets = [sheet for sheet in wb.sheet_names
                      if sheet not in ['Information', 'iSAMS Information', 'Sheet1']
                      and not sheet.startswith('Sheet')]

        if not class_sheets:
            print("No class sheets found in the template.")
            return

        # Process each class sheet
        all_dfs = {}
        for sheet in class_sheets:
            # Read the sheet
            df = pd.read_excel(io.BytesIO(content), sheet_name=sheet, header=None)

            # Find the row with student data
            student_row_idx = None
            for i in range(len(df)):
                if isinstance(df.iloc[i, 0], str) and 'SchoolID' in df.iloc[i, 0]:
                    student_row_idx = i
                    break

            if student_row_idx is None:
                print(f"Could not find student data row in sheet {sheet}.")
                continue

            # Set the header row
            df.columns = df.iloc[student_row_idx]
            df = df.iloc[student_row_idx + 1:].reset_index(drop=True)

            # Find where the Comment column is
            comment_col = None
            for col in df.columns:
                if isinstance(col, str) and 'Comment' in col:
                    comment_col = col
                    break

            if comment_col is None:
                print(f"Could not find Comment column in sheet {sheet}.")
                continue

            # Look for the SchoolID or identifier column
            id_col = None
            for col in ['SchoolID', 'tblReportStoreID']:
                if col in df.columns:
                    id_col = col
                    break

            if id_col is None:
                print(f"Could not find ID column in sheet {sheet}.")
                continue

            # Update the Comments column with our generated comments
            for idx, row in df.iterrows():
                # Check if we have a comment for this student
                student_found = False
                school_id = row[id_col]

                # Try to match by SchoolID or by name
                for student_id, comment_data in student_comments.items():
                    if (pd.notna(school_id) and pd.notna(comment_data.get('SchoolID')) and
                        str(school_id) == str(comment_data.get('SchoolID'))):
                        # Match by SchoolID
                        df.at[idx, comment_col] = comment_data['Comment']
                        student_found = True
                        break
                    elif (pd.notna(row.get('Surname')) and pd.notna(row.get('Forename')) and
                          row['Surname'] == comment_data['Surname'] and
                          row['Forename'] == comment_data['Forename']):
                        # Match by name
                        df.at[idx, comment_col] = comment_data['Comment']
                        student_found = True
                        break

            all_dfs[sheet] = df

        # Create a writer
        output = io.BytesIO()
        with pd.ExcelWriter(output, engine='openpyxl') as writer:
            # Copy sheets from the template
            wb_obj = pd.ExcelFile(io.BytesIO(content))
            all_sheets = wb_obj.sheet_names

            for sheet in all_sheets:
                if sheet in all_dfs:
                    # This is a sheet we modified
                    all_dfs[sheet].to_excel(writer, sheet_name=sheet, index=False, header=True)
                else:
                    # Copy the original sheet
                    pd.read_excel(io.BytesIO(content), sheet_name=sheet).to_excel(
                        writer, sheet_name=sheet, index=False)

        # Save the file
        output.seek(0)

        # Create a filename
        export_filename = f"iSAMS_Export_{selected_teacher.replace('(', '').replace(')', '')}_{selected_term}.xlsx"

        # Download the file
        with open(export_filename, 'wb') as f:
            f.write(output.getvalue())

        files.download(export_filename)

        print(f"Export completed successfully! File: {export_filename}")

    except Exception as e:
        print(f"Error during export: {str(e)}")

In [12]:
# Start the application
def start_app():
    upload_student_data()

# Run the app
start_app()

Please upload the student data Excel file...


Saving 130924-PRIMARY-SETS-ISAMS-EXPORT.xlsx to 130924-PRIMARY-SETS-ISAMS-EXPORT (2).xlsx
Loaded 842 student records.

First 5 records:


,Surname,Forename (Firstname),Preferred Name,Form,Year Group (NC),Academic House,Gender,Teacher,Set
0,Abousharkh,Ryan,Ryan,Y3/C,3,Falcon,M,(JGI),3x-PEC-BFL
1,Ahli,Khalid Ahmad Hassan Baqer,NaN,Y3/B,3,Leopard,M,(JGI),3x-PEC-BFL
2,Al Shamsi,Mohammed,Mohammed,Y3/C,3,Falcon,M,(JGI),3x-PEC-BFL
3,Aql,Basel Amin,NaN,Y3/A,3,Falcon,M,(JGI),3x-PEC-BFL
4,Beser,Derin,NaN,Y3/B,3,Falcon,M,(JGI),3x-PEC-BFL


Dropdown(description='Select Teacher:', options=('(JGI)', '(MAC)', '(CFR)', '(JGB)', '(DTH)', '(ASM)', '(PSU)'…


Selected teacher: (DTH)
This teacher has 118 students.
Created sample comment segments for testing:


,Category,Code,Text,Level,Unit
0,Intro,INT01,[First_Name] has made a good start to the term...,1,General
1,Intro,INT02,[First_Name] has shown excellent progress in P...,2,General
2,Effort,EFF01,[He/She] consistently puts forth good effort i...,1,General
3,Effort,EFF02,[First_Name] demonstrates enthusiasm and commi...,2,General
4,Attitude,ATT01,[He/She] has a positive attitude toward learni...,1,General
5,Attitude,ATT02,[First_Name] shows determination when facing c...,2,General
6,Closing,CLS01,I am pleased with [his/her] progress and look ...,1,General
7,Closing,CLS02,[First_Name] should continue to practice regul...,2,General



Added sport-specific comments:


,Category,Code,Text,Level,Unit
8,Swimming,SWM01,[First_Name] is developing confidence in the w...,1,Swimming
9,Swimming,SWM02,[He/She] can swim independently and is learnin...,2,Swimming
10,Swimming,SWM03,[First_Name] shows excellent technique in [his...,3,Swimming
11,Football,FTB01,[First_Name] is learning the basic rules and s...,1,Football
12,Football,FTB02,[He/She] participates well in team games and i...,2,Football
13,Football,FTB03,[First_Name] shows talent in football and demo...,3,Football


NameError: name 'show_term_selection' is not defined